In [15]:
%load_ext autoreload
%autoreload 2

import torch
import numpy as np
import model, ode, optimize, net, tools

import gpytorch
import torch.nn as nn

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [62]:
x = torch.linspace(-1,1,50)
y = torch.linspace(-1,1,50)
t = torch.linspace(0,1,6)

X,Y = torch.meshgrid(x,y,indexing='xy')
XY = torch.stack([X,Y], dim = -1).reshape(-1,2)
T = t


In [63]:
def vel(t, xy, A=1.0, omega= 2 * 3.14159, epsilon=0.25):
    x = xy[..., 0]  + 0.5
    y = xy[..., 1]

    a = epsilon * torch.sin(torch.tensor(0))
    b = 1 - 2 * a  

    f = a * x**2 + b * x
    df_dx = 2 * a * x + b

    u = -3.14159 * A * torch.sin(3.14159 * f) * torch.cos(3.14159 * y)
    v = 3.14159 * A * torch.cos(3.14159 * f) * torch.sin(3.14159 * y) * df_dx

    return torch.stack([u, v], dim=-1)/10

flow = ode.Flow(vel)


In [68]:
kernel = gpytorch.kernels.ScaleKernel(
    gpytorch.kernels.MaternKernel(nu = 5/2, ard_num_dims = 3))
likelihood = gpytorch.likelihoods.GaussianLikelihood()
likelihood.noise = 0.01
hypers = {
    'base_kernel.lengthscale': torch.tensor([1.0, 0.2, 0.2]),
    'outputscale': torch.tensor(1),
    }
kernel.initialize(**hypers);

In [69]:
gp = model.GP(kernel, likelihood)

In [70]:
import simulate
Z = simulate.observations(gp, flow, T, XY)

In [89]:
data = model.data(T,XY,Z)
data.indices = tools.point_sampling(data.XY, min_dist=0.01)

In [90]:
data.indices.shape[0]

1000

In [91]:
data.plot_observations(data.indices)

In [12]:
kernel = gpytorch.kernels.ScaleKernel(
    gpytorch.kernels.MaternKernel(nu = 5/2, ard_num_dims = 3))
likelihood = gpytorch.likelihoods.GaussianLikelihood()
hypers = {
    'base_kernel.lengthscale': torch.tensor([0.25, 0.2, 0.2]),
    'outputscale': torch.tensor(1),
    }
kernel.initialize(**hypers);

In [13]:
gp = model.GP(kernel, likelihood)

In [14]:
optimize.gp(gp, data, num_epochs = 200)

Epoch: 25 - Likelihood: 0.009
Epoch: 50 - Likelihood: -0.890
Epoch: 75 - Likelihood: -1.428
Epoch: 100 - Likelihood: -1.619
Epoch: 125 - Likelihood: -1.671
Epoch: 150 - Likelihood: -1.690
Epoch: 175 - Likelihood: -1.700
Epoch: 200 - Likelihood: -1.705


In [16]:
flow = net.Flow(L = 5) 

In [17]:
optimize.flow_vecchia(gp, flow, data, num_epochs=100)

Epoch: 10 - Likelihood: -1.643
Epoch: 20 - Likelihood: -2.010
Epoch: 30 - Likelihood: -2.187
Epoch: 40 - Likelihood: -2.291
Epoch: 50 - Likelihood: -2.362
Epoch: 60 - Likelihood: -2.382
Epoch: 70 - Likelihood: -2.416
Epoch: 80 - Likelihood: -2.440
Epoch: 90 - Likelihood: -2.431
Epoch: 100 - Likelihood: -2.467


In [18]:
gp.kernel.base_kernel.lengthscale

tensor([[3.4402, 0.9782, 0.9486]], grad_fn=<SoftplusBackward0>)

In [19]:
T_ = data.T
XY_ = data.XY
sub = 1200
indices = torch.randperm(data.m)[:sub]
m = indices.shape[0]

UV = torch.ones((T_.shape[0], XY_.shape[0], 2 ))
for frame in range(0,T_.shape[0]):
    UV[frame,:,:] = vel(T_[frame], XY_)

XY_UV = [torch.cat([XY_,
                    UV[i,:,0:1],
                    UV[i,:,1:2]],
        dim = -1).detach() for i in range(0,T_.shape[0])]

In [20]:
data.XY_UV = XY_UV


In [21]:
data.vel = ode.Vel_hat(flow, data)

In [23]:
sub = data.m//2
indices = torch.randperm(data.m)[:sub]
data.plot_vel(indices)

In [46]:
data.Z.shape

torch.Size([6, 2500])

In [ ]:
[Z for Z in data.Z]